# * VINSIGHT : Data Issue

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Any Issue History

### Prepaid Active Caller : TMH

In [2]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250401
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'TB1S000600' --Prepaid Active Caller : TMH
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    # print(f'\nCurrent Timestamp...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    # txn_df = txn_df.fillna(0, inplace=True)
    # txn_df = txn_df.replace(np.nan, None)
    mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-02, 10:40:45
   -> v_start_date: 20250401

    TM_KEY_MTH  TM_KEY_DAY   METRIC_CD                              METRIC_NAME   AREA_NAME  ACTUAL_AS_OF ACTUAL_SNAP ACTUAL_AGG_MTH              PPN_TM
0       202504    20250401  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250401  17,205,721     17,205,721 2025-05-02 08:51:18
1       202504    20250402  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250402  17,200,601     17,200,601 2025-05-02 08:51:18
2       202504    20250403  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250403  17,198,038     17,198,038 2025-05-02 08:51:18
3       202504    20250404  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250404  17,166,314     17,166,314 2025-05-02 08:51:18
4       202504    20250405  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250405  17,132,397     17,132,397 2025-05-02 08

### Area 080 : DTAC

In [12]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME
        , SUM(CASE WHEN AREA_CD = '080' THEN ACTUAL_SNAP END) ACTUAL_080
        , SUM(CASE WHEN AREA_CD = '080' THEN TARGET_SNAP END) TARGET_080
        , SUM(CASE WHEN AREA_CD = '016' THEN ACTUAL_SNAP END) ACTUAL_016
        , SUM(CASE WHEN AREA_CD = '016' THEN TARGET_SNAP END) TARGET_016
        , SUM(CASE WHEN AREA_CD = '040' THEN ACTUAL_SNAP END) ACTUAL_040
        , SUM(CASE WHEN AREA_CD = '040' THEN TARGET_SNAP END) TARGET_040
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF, MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'DB1R000100' --Prepaid Revenue : DTAC
    --WHERE METRIC_CD = 'DB2R000100' --Postpaid Revenue : DTAC
    --WHERE METRIC_CD = 'DB2R010100' --Postpaid Revenue B2C : DTAC
    --WHERE METRIC_CD = 'DB2R020100' --Postpaid Revenue B2B : DTAC
    AND AREA_CD IN ('016', '040', '080')
    AND TM_KEY_DAY >= {v_start_date} 
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME
    ORDER BY TM_KEY_DAY, METRIC_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    txn_df = txn_df.replace(np.nan, 0)
    mod_col_list = txn_df.iloc[:, 4:10].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=100)}')
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-02, 10:49:54
   -> v_start_date: 20250101


C:\Users\Narut4\AppData\Local\Temp\ipykernel_12452\633590385.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  txn_df = txn_df.replace(np.nan, 0)



     TM_KEY_MTH  TM_KEY_DAY   METRIC_CD             METRIC_NAME ACTUAL_080 TARGET_080 ACTUAL_016 TARGET_016 ACTUAL_040 TARGET_040  ACTUAL_AS_OF              PPN_TM
0        202501    20250101  DB1R000100  Prepaid Revenue : DTAC  1,458,637          0          0          0          0          0      20250101 2025-05-01 00:08:03
1        202501    20250102  DB1R000100  Prepaid Revenue : DTAC  1,378,370          0          0          0          0          0      20250102 2025-05-01 00:08:03
2        202501    20250103  DB1R000100  Prepaid Revenue : DTAC  1,330,136          0          0          0          0          0      20250103 2025-05-01 00:08:03
3        202501    20250104  DB1R000100  Prepaid Revenue : DTAC  1,229,670          0          0          0          0          0      20250104 2025-05-01 00:08:03
4        202501    20250105  DB1R000100  Prepaid Revenue : DTAC  1,312,542          0          0          0          0          0      20250105 2025-05-01 00:08:03
5        202501

### Prepaid Revenue : TMH

In [13]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250401
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'TB1R000100' --Prepaid Revenue : TMH
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    # txn_df = txn_df.replace(np.nan, 0)
    mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-02, 10:50:20
   -> v_start_date: 20250401

    TM_KEY_MTH  TM_KEY_DAY   METRIC_CD            METRIC_NAME   AREA_NAME  ACTUAL_AS_OF  ACTUAL_SNAP ACTUAL_AGG_MTH              PPN_TM
0       202504    20250401  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250401  101,725,829    101,725,829 2025-05-02 08:51:18
1       202504    20250402  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250402   92,314,263    194,040,092 2025-05-02 08:51:18
2       202504    20250403  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250403   85,596,819    279,636,911 2025-05-02 08:51:18
3       202504    20250404  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250404   85,954,445    365,591,356 2025-05-02 08:51:18
4       202504    20250405  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250405   84,406,717    449,998,074 2025-05-02 08:51:18
5       202504    20250406  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250406   80,94

### Monthly Revenue ARPU

In [14]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B7R000102' --Prepaid ARPU
        , 'DB7R000102' --Prepaid ARPU : DTAC
        , 'TB7R000102' --Prepaid ARPU : TMH
        , 'B7R000101' --Postpaid ARPU
        , 'B7R010101' --Postpaid ARPU B2C
        , 'DB7R000101' --Postpaid ARPU : DTAC
        , 'DB7R010101' --Postpaid ARPU B2C : DTAC
        , 'TB7R000101' --Postpaid ARPU : TMH
        , 'TB7R010101' --Postpaid ARPU B2C : TMH
        , 'TB7R000103' --TOL ARPU
        , 'TB7R000104' --TVS ARPU
        )
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    --ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # # Display
    # txn_df = chk_src_df.copy()
    # mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    # for col in mod_col_list:
    #     txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    # print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-02, 10:50:51
   -> v_start_date: 20250101


In [15]:
''' B7R000102 : Prepaid ARPU '''

v_metric_cd = 'B7R000102'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nB7R000102 : Prepaid ARPU...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B7R000102_day_df = chk_src_df.copy()
B7R000102_day_df = B7R000102_day_df.loc[B7R000102_day_df['METRIC_CD']==v_metric_cd]
# B7R000102_day_df = B7R000102_day_df.loc[B7R000102_day_df['TM_KEY_DAY']>=v_tm_key_day]
B7R000102_day_df = B7R000102_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = B7R000102_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    B7R000102_day_df[col] = B7R000102_day_df[col].apply(lambda x: format(x, ',.1f'))
    
B7R000102_day_df


B7R000102 : Prepaid ARPU...

   -> v_metric_cd: B7R000102


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,B7R000102,Prepaid ARPU,Nationwide,20250101,172.9,172.9,2025-05-02 08:51:18
1,202501,20250102,B7R000102,Prepaid ARPU,Nationwide,20250102,172.8,172.8,2025-05-02 08:51:18
2,202501,20250103,B7R000102,Prepaid ARPU,Nationwide,20250103,173.2,173.2,2025-05-02 08:51:18
3,202501,20250104,B7R000102,Prepaid ARPU,Nationwide,20250104,173.4,173.4,2025-05-02 08:51:18
4,202501,20250105,B7R000102,Prepaid ARPU,Nationwide,20250105,173.5,173.5,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,B7R000102,Prepaid ARPU,Nationwide,20250427,151.5,151.5,2025-05-02 08:51:18
117,202504,20250428,B7R000102,Prepaid ARPU,Nationwide,20250428,151.5,151.5,2025-05-02 08:51:18
118,202504,20250429,B7R000102,Prepaid ARPU,Nationwide,20250428,0.0,151.5,2025-05-02 08:51:18
119,202504,20250430,B7R000102,Prepaid ARPU,Nationwide,20250428,0.0,151.5,2025-05-02 08:51:18


In [16]:
''' B7R010101 : Postpaid ARPU B2C '''

v_metric_cd = 'B7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nB7R010101 : Postpaid ARPU B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B7R010101_day_df = chk_src_df.copy()
B7R010101_day_df = B7R010101_day_df.loc[B7R010101_day_df['METRIC_CD']==v_metric_cd]
# B7R010101_day_df = B7R010101_day_df.loc[B7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
B7R010101_day_df = B7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = B7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    B7R010101_day_df[col] = B7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
B7R010101_day_df


B7R010101 : Postpaid ARPU B2C...

   -> v_metric_cd: B7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,B7R010101,Postpaid ARPU B2C,Nationwide,20250101,416.8,416.8,2025-05-02 08:51:18
1,202501,20250102,B7R010101,Postpaid ARPU B2C,Nationwide,20250102,417.0,417.0,2025-05-02 08:51:18
2,202501,20250103,B7R010101,Postpaid ARPU B2C,Nationwide,20250103,417.0,417.0,2025-05-02 08:51:18
3,202501,20250104,B7R010101,Postpaid ARPU B2C,Nationwide,20250104,417.0,417.0,2025-05-02 08:51:18
4,202501,20250105,B7R010101,Postpaid ARPU B2C,Nationwide,20250105,416.7,416.7,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,B7R010101,Postpaid ARPU B2C,Nationwide,20250427,416.7,416.7,2025-05-02 08:51:18
117,202504,20250428,B7R010101,Postpaid ARPU B2C,Nationwide,20250428,392.3,392.3,2025-05-02 08:51:18
118,202504,20250429,B7R010101,Postpaid ARPU B2C,Nationwide,20250428,0.0,392.3,2025-05-02 08:51:18
119,202504,20250430,B7R010101,Postpaid ARPU B2C,Nationwide,20250428,0.0,392.3,2025-05-02 08:51:18


In [17]:
''' TB7R010101 : Postpaid ARPU B2C : TMH '''

v_metric_cd = 'TB7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R010101 : Postpaid ARPU B2C : TMH...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R010101_day_df = chk_src_df.copy()
TB7R010101_day_df = TB7R010101_day_df.loc[TB7R010101_day_df['METRIC_CD']==v_metric_cd]
# TB7R010101_day_df = TB7R010101_day_df.loc[TB7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R010101_day_df = TB7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R010101_day_df[col] = TB7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R010101_day_df


TB7R010101 : Postpaid ARPU B2C : TMH...

   -> v_metric_cd: TB7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250101,371.6,371.6,2025-05-02 08:51:18
1,202501,20250102,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250102,372.0,372.0,2025-05-02 08:51:18
2,202501,20250103,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250103,372.0,372.0,2025-05-02 08:51:18
3,202501,20250104,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250104,372.0,372.0,2025-05-02 08:51:18
4,202501,20250105,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250105,371.9,371.9,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250427,371.7,371.7,2025-05-02 08:51:18
117,202504,20250428,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250428,333.7,333.7,2025-05-02 08:51:18
118,202504,20250429,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250429,333.9,333.9,2025-05-02 08:51:18
119,202504,20250430,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250429,0.0,333.9,2025-05-02 08:51:18


In [18]:
''' DB7R010101 : Postpaid ARPU B2C : DTAC '''

v_metric_cd = 'DB7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nDB7R010101 : Postpaid ARPU B2C : DTAC...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

DB7R010101_day_df = chk_src_df.copy()
DB7R010101_day_df = DB7R010101_day_df.loc[DB7R010101_day_df['METRIC_CD']==v_metric_cd]
# DB7R010101_day_df = DB7R010101_day_df.loc[DB7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
DB7R010101_day_df = DB7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = DB7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    DB7R010101_day_df[col] = DB7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
DB7R010101_day_df


DB7R010101 : Postpaid ARPU B2C : DTAC...

   -> v_metric_cd: DB7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250101,494.7,494.7,2025-05-02 08:51:18
1,202501,20250102,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250102,494.5,494.5,2025-05-02 08:51:18
2,202501,20250103,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250103,494.6,494.6,2025-05-02 08:51:18
3,202501,20250104,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250104,494.4,494.4,2025-05-02 08:51:18
4,202501,20250105,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250105,494.1,494.1,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250427,496.4,496.4,2025-05-02 08:51:18
117,202504,20250428,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250428,496.2,496.2,2025-05-02 08:51:18
118,202504,20250429,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250428,0.0,496.2,2025-05-02 08:51:18
119,202504,20250430,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250428,0.0,496.2,2025-05-02 08:51:18


In [19]:
''' TB7R000103 : TOL ARPU '''

v_metric_cd = 'TB7R000103'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R000103 : TOL ARPU...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R000103_day_df = chk_src_df.copy()
TB7R000103_day_df = TB7R000103_day_df.loc[TB7R000103_day_df['METRIC_CD']==v_metric_cd]
# TB7R000103_day_df = TB7R000103_day_df.loc[TB7R000103_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R000103_day_df = TB7R000103_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R000103_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R000103_day_df[col] = TB7R000103_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R000103_day_df


TB7R000103 : TOL ARPU...

   -> v_metric_cd: TB7R000103


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R000103,TOL ARPU,Nationwide,20250101,510.9,510.9,2025-05-02 08:51:18
1,202501,20250102,TB7R000103,TOL ARPU,Nationwide,20250102,510.8,510.8,2025-05-02 08:51:18
2,202501,20250103,TB7R000103,TOL ARPU,Nationwide,20250103,510.9,510.9,2025-05-02 08:51:18
3,202501,20250104,TB7R000103,TOL ARPU,Nationwide,20250104,510.8,510.8,2025-05-02 08:51:18
4,202501,20250105,TB7R000103,TOL ARPU,Nationwide,20250105,510.6,510.6,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,TB7R000103,TOL ARPU,Nationwide,20250427,514.3,514.3,2025-05-02 08:51:18
117,202504,20250428,TB7R000103,TOL ARPU,Nationwide,20250428,513.3,513.3,2025-05-02 08:51:18
118,202504,20250429,TB7R000103,TOL ARPU,Nationwide,20250428,0.0,513.3,2025-05-02 08:51:18
119,202504,20250430,TB7R000103,TOL ARPU,Nationwide,20250428,0.0,513.3,2025-05-02 08:51:18


In [20]:
''' TB7R000104 : TVS ARPU '''

v_metric_cd = 'TB7R000104'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R000104 : Postpaid ARPU B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R000104_day_df = chk_src_df.copy()
TB7R000104_day_df = TB7R000104_day_df.loc[TB7R000104_day_df['METRIC_CD']==v_metric_cd]
# TB7R000104_day_df = TB7R000104_day_df.loc[TB7R000104_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R000104_day_df = TB7R000104_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R000104_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R000104_day_df[col] = TB7R000104_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R000104_day_df


TB7R000104 : Postpaid ARPU B2C...

   -> v_metric_cd: TB7R000104


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R000104,TVS ARPU,Nationwide,20250101,260.4,260.4,2025-05-02 08:51:18
1,202501,20250102,TB7R000104,TVS ARPU,Nationwide,20250102,259.8,259.8,2025-05-02 08:51:18
2,202501,20250103,TB7R000104,TVS ARPU,Nationwide,20250103,259.8,259.8,2025-05-02 08:51:18
3,202501,20250104,TB7R000104,TVS ARPU,Nationwide,20250104,259.9,259.9,2025-05-02 08:51:18
4,202501,20250105,TB7R000104,TVS ARPU,Nationwide,20250105,259.9,259.9,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,TB7R000104,TVS ARPU,Nationwide,20250427,259.0,259.0,2025-05-02 08:51:18
117,202504,20250428,TB7R000104,TVS ARPU,Nationwide,20250428,258.0,258.0,2025-05-02 08:51:18
118,202504,20250429,TB7R000104,TVS ARPU,Nationwide,20250428,0.0,258.0,2025-05-02 08:51:18
119,202504,20250430,TB7R000104,TVS ARPU,Nationwide,20250428,0.0,258.0,2025-05-02 08:51:18


## * Revenue Corporate (C, P)

In [67]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-02, 16:54:22
   -> v_start_date: 20250101


In [103]:
sample_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==20250102].copy()

mod_col_list = sample_df.iloc[:, 7:12].columns.tolist()
for col in mod_col_list:
    sample_df[col] = sample_df[col].apply(lambda x: format(x, ',.0f'))

sample_df = sample_df.sort_values(by=['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD']).reset_index(drop=True)
sample_df

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,ACTUAL_AS_OF,C,P,G,H,HH,PPN_TM
0,202501,20250102,All Services,ALL,B0R000100,Total Revenue,20250102.0,"641,115,933","1,253,584,807",nan,nan,nan,2025-05-02 08:51:18
1,202501,20250102,All Services,DTAC,DB0R000100,Total Revenue : DTAC,20250102.0,"301,030,841","301,035,463","301,035,463","300,771,410","300,771,410",2025-05-02 15:37:10
2,202501,20250102,All Services,TRUE,TB0R000100,Total Revenue : TRUE,20250102.0,"340,085,092","952,549,345","335,844,364","335,769,394","335,767,827",2025-05-02 15:37:10
3,202501,20250102,Mobile,ALL,B0R000101,Mobile Revenue,20250102.0,"389,541,647","1,003,267,419","386,562,438","386,298,101","386,298,101",2025-05-02 15:37:10
4,202501,20250102,Mobile,DTAC,DB0R000101,Mobile Revenue : DTAC,20250102.0,"301,030,841","301,035,463","301,035,463","300,771,410","300,771,410",2025-05-02 15:37:10
5,202501,20250102,Mobile,TRUE,TB0R000101,Mobile Revenue : TMH,20250102.0,"88,510,806","702,231,956","85,526,976","85,526,690","85,526,690",2025-05-02 15:37:10
6,202501,20250102,Postpaid,ALL,B2R000100,Postpaid Revenue,20250102.0,"847,437,062","847,437,062","233,715,912","233,452,187","233,452,187",2025-05-02 15:37:10
7,202501,20250102,Postpaid,ALL,B2R010100,Postpaid Revenue B2C,20250102.0,"577,284,228","577,284,228","577,283,416","577,281,093","577,281,093",2025-05-02 15:37:10
8,202501,20250102,Postpaid,DTAC,DB2R000100,Postpaid Revenue : DTAC,20250102.0,"233,715,912","233,715,912","233,715,912","233,452,187","233,452,187",2025-05-02 15:37:10
9,202501,20250102,Postpaid,DTAC,DB2R010100,Postpaid Revenue B2C : DTAC,20250102.0,"97,527,120","97,527,120","97,527,120","97,526,880","97,526,880",2025-05-02 15:37:10


In [97]:
sample_df2 = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()

sample_df2['METRIC_TMP'] = sample_df2['METRIC_NAME'].replace(r' : DTAC| : TRUE| : TMH', '', regex=True)
sample_df2['ALL'] = np.where(sample_df2['COMP_CD']=='ALL', sample_df2['C'], 0)
sample_df2['TRUE'] = np.where(sample_df2['COMP_CD']=='TRUE', sample_df2['C'], 0)
sample_df2['DTAC'] = np.where(sample_df2['COMP_CD']=='DTAC', sample_df2['C'], 0)

sample_df2 = sample_df2.groupby(['TM_KEY_MTH', 'METRIC_TMP']).agg({'ACTUAL_AS_OF':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum', 'PPN_TM':'max'})
sample_df2 = sample_df2.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_TMP']).reset_index()

mod_col_list = sample_df2.iloc[:, 3:6].columns.tolist()
for col in mod_col_list:
    sample_df2[col] = sample_df2[col].apply(lambda x: format(x, ',.0f'))

sample_df2

,TM_KEY_MTH,METRIC_TMP,ACTUAL_AS_OF,ALL,TRUE,DTAC,PPN_TM
0,202501,Mobile Revenue,20250131.0,"11,520,556,591","2,444,926,240","9,075,630,351",2025-05-02 15:37:10
1,202501,Postpaid Revenue,20250131.0,"6,470,994,119","3,731,079,534","2,739,914,585",2025-05-02 15:37:10
2,202501,Postpaid Revenue B2C,20250131.0,"5,868,396,046","3,336,770,490","2,531,625,556",2025-05-02 15:37:10
3,202501,Prepaid Revenue,20250131.0,"6,020,739,419","2,444,926,240","3,575,813,178",2025-05-02 15:37:10
4,202501,TOL Revenue,20250128.0,0,"1,597,056,733",0,2025-05-02 15:37:10
5,202501,TVS Revenue,20250131.0,0,"417,316,338",0,2025-05-02 15:37:10
6,202501,Total Revenue,20250131.0,"13,534,929,662","4,459,299,311","9,075,630,351",2025-05-02 15:37:10


In [89]:
''' Total Revenue '''

v_total_metric_list = ['B0R000100', 'TB0R000100', 'DB0R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTotal Revenue...\n')
print(f'   -> v_total_metric_list: {v_total_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B0R000100_day_df = chk_src_df.copy()
B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['METRIC_CD'].isin(v_total_metric_list)]
# B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

# B0R000100_day_df['METRIC_TMP'] = B0R000100_day_df['METRIC_NAME'].replace(r' : DTAC| : TRUE| : TMH', '', regex=True)
# B0R000100_day_df['ALL'] = np.where(B0R000100_day_df['COMP_CD']=='ALL', B0R000100_day_df['ACTUAL_SNAP'], 0)
# B0R000100_day_df['TRUE'] = np.where(B0R000100_day_df['COMP_CD']=='TRUE', B0R000100_day_df['ACTUAL_SNAP'], 0)
# B0R000100_day_df['DTAC'] = np.where(B0R000100_day_df['COMP_CD']=='DTAC', B0R000100_day_df['ACTUAL_SNAP'], 0)

B0R000100_day_df = B0R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
B0R000100_day_df = B0R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# B0R000100_day_df['ACTUAL_AS_OF'] = B0R000100_day_df['ACTUAL_AS_OF'].astype(str)

B0R000100_day_df['Diff (C - P)'] = B0R000100_day_df['C'] - B0R000100_day_df['P']
B0R000100_day_df = B0R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = B0R000100_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    B0R000100_day_df[col] = B0R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B0R000100_day_df


Total Revenue...

   -> v_total_metric_list: ['B0R000100', 'TB0R000100', 'DB0R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,All Services,B0R000100,Total Revenue,"893,588,001","13,534,929,662","12,641,341,660",0,2025-05-02 08:51:18
1,202501,All Services,DB0R000100,Total Revenue : DTAC,"4,495,688,784","9,075,630,351","4,579,941,567","4,579,940,849",2025-05-02 15:37:10
2,202501,All Services,TB0R000100,Total Revenue : TRUE,"-3,602,100,783","4,459,299,311","8,061,400,094","4,316,603,800",2025-05-02 15:37:10
3,202502,All Services,B0R000100,Total Revenue,"528,241,490","12,479,558,362","11,951,316,872",0,2025-05-02 08:51:18
4,202502,All Services,DB0R000100,Total Revenue : DTAC,"4,236,205,881","8,347,773,941","4,111,568,059","4,111,566,862",2025-05-02 15:37:10
5,202502,All Services,TB0R000100,Total Revenue : TRUE,"-3,707,964,391","4,131,784,421","7,839,748,812","4,065,402,793",2025-05-02 15:37:10
6,202503,All Services,B0R000100,Total Revenue,"684,291,842","13,272,113,230","12,587,821,388",0,2025-05-02 08:51:18
7,202503,All Services,DB0R000100,Total Revenue : DTAC,"4,471,281,027","8,981,683,086","4,510,402,059","4,510,400,962",2025-05-02 15:37:10
8,202503,All Services,TB0R000100,Total Revenue : TRUE,"-3,786,989,185","4,290,430,144","8,077,419,329","4,300,343,855",2025-05-02 15:37:10
9,202504,All Services,B0R000100,Total Revenue,"-3,712,503,397","8,552,217,541","12,264,720,938",0,2025-05-02 08:51:18


In [90]:
''' Mobile Revenue '''

v_mobile_metric_list = ['B0R000101', 'TB0R000101', 'DB0R000101']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nMobile Revenue...\n')
print(f'   -> v_mobile_metric_list: {v_mobile_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B0R000101_day_df = chk_src_df.copy()
B0R000101_day_df = B0R000101_day_df.loc[B0R000101_day_df['METRIC_CD'].isin(v_mobile_metric_list)]
# B0R000101_day_df = B0R000101_day_df.loc[B0R000101_day_df['TM_KEY_MTH']==v_tm_key_mth]
# B0R000101_day_df = B0R000101_day_df.loc[B0R000101_day_df['TM_KEY_DAY']>=v_tm_key_day]

B0R000101_day_df = B0R000101_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
B0R000101_day_df = B0R000101_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

B0R000101_day_df['Diff (C - P)'] = B0R000101_day_df['C'] - B0R000101_day_df['P']
B0R000101_day_df = B0R000101_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = B0R000101_day_df.iloc[:, 4:7].columns.tolist()
for col in mod_col_list:
    B0R000101_day_df[col] = B0R000101_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B0R000101_day_df


Mobile Revenue...

   -> v_mobile_metric_list: ['B0R000101', 'TB0R000101', 'DB0R000101']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,Mobile,B0R000101,Mobile Revenue,"764,609,250","11,520,556,591","10,755,947,341",7.011150e+09,2025-05-02 15:37:10
1,202501,Mobile,DB0R000101,Mobile Revenue : DTAC,"4,495,688,784","9,075,630,351","4,579,941,567",4.579941e+09,2025-05-02 15:37:10
2,202501,Mobile,TB0R000101,Mobile Revenue : TMH,"-3,731,079,534","2,444,926,240","6,176,005,774",2.431209e+09,2025-05-02 15:37:10
3,202502,Mobile,B0R000101,Mobile Revenue,"509,774,943","10,562,656,620","10,052,881,677",6.278534e+09,2025-05-02 15:37:10
4,202502,Mobile,DB0R000101,Mobile Revenue : DTAC,"4,236,205,881","8,347,773,941","4,111,568,059",4.111567e+09,2025-05-02 15:37:10
5,202502,Mobile,TB0R000101,Mobile Revenue : TMH,"-3,726,430,939","2,214,882,680","5,941,313,618",2.166968e+09,2025-05-02 15:37:10
6,202503,Mobile,B0R000101,Mobile Revenue,"762,727,107","11,441,052,542","10,678,325,435",6.901249e+09,2025-05-02 15:37:10
7,202503,Mobile,DB0R000101,Mobile Revenue : DTAC,"4,471,281,027","8,981,683,086","4,510,402,059",4.510401e+09,2025-05-02 15:37:10
8,202503,Mobile,TB0R000101,Mobile Revenue : TMH,"-3,708,553,920","2,459,369,456","6,167,923,376",2.390848e+09,2025-05-02 15:37:10
9,202504,Mobile,B0R000101,Mobile Revenue,"-3,729,651,462","6,720,194,090","10,449,845,552",6.648235e+09,2025-05-02 15:37:10


In [91]:
''' Prepaid Revenue '''

v_prepaid_metric_list = ['B1R000100', 'TB1R000100', 'DB1R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nPrepaid Revenue...\n')
print(f'   -> v_prepaid_metric_list: {v_prepaid_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B1R000100_day_df = chk_src_df.copy()
B1R000100_day_df = B1R000100_day_df.loc[B1R000100_day_df['METRIC_CD'].isin(v_prepaid_metric_list)]
# B1R000100_day_df = B1R000100_day_df.loc[B1R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# B1R000100_day_df = B1R000100_day_df.loc[B1R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

B1R000100_day_df = B1R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
B1R000100_day_df = B1R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

B1R000100_day_df['Diff (C - P)'] = B1R000100_day_df['C'] - B1R000100_day_df['P']
B1R000100_day_df = B1R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = B1R000100_day_df.iloc[:, 4:7].columns.tolist()
for col in mod_col_list:
    B1R000100_day_df[col] = B1R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B1R000100_day_df


Prepaid Revenue...

   -> v_prepaid_metric_list: ['B1R000100', 'TB1R000100', 'DB1R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,Prepaid,B1R000100,Prepaid Revenue,"1,735,786,197","6,020,739,419","4,284,953,222",4.271236e+09,2025-05-02 15:37:10
1,202501,Prepaid,DB1R000100,Prepaid Revenue : DTAC,"1,735,786,197","3,575,813,178","1,840,026,982",1.840027e+09,2025-05-02 15:37:10
2,202501,Prepaid,TB1R000100,Prepaid Revenue : TMH,0,"2,444,926,240","2,444,926,240",2.431209e+09,2025-05-02 15:37:10
3,202502,Prepaid,B1R000100,Prepaid Revenue,"1,570,874,962","5,434,575,793","3,863,700,831",3.815786e+09,2025-05-02 15:37:10
4,202502,Prepaid,DB1R000100,Prepaid Revenue : DTAC,"1,570,874,962","3,219,693,113","1,648,818,151",1.648818e+09,2025-05-02 15:37:10
5,202502,Prepaid,TB1R000100,Prepaid Revenue : TMH,0,"2,214,882,680","2,214,882,680",2.166968e+09,2025-05-02 15:37:10
6,202503,Prepaid,B1R000100,Prepaid Revenue,"1,736,475,223","5,971,278,728","4,234,803,506",4.166282e+09,2025-05-02 15:37:10
7,202503,Prepaid,DB1R000100,Prepaid Revenue : DTAC,"1,736,475,223","3,511,909,273","1,775,434,050",1.775434e+09,2025-05-02 15:37:10
8,202503,Prepaid,TB1R000100,Prepaid Revenue : TMH,0,"2,459,369,456","2,459,369,456",2.390848e+09,2025-05-02 15:37:10
9,202504,Prepaid,B1R000100,Prepaid Revenue,"-11,957","3,975,563,898","3,975,575,855",3.903608e+09,2025-05-02 15:37:10


In [101]:
# ''' Postpaid Revenue '''

# v_postpaid_metric_list = ['B2R000100', 'TB2R000100', 'DB2R000100']
# # v_tm_key_mth = 202501
# # v_tm_key_day = int(str(curr_mth)+str('01'))
# # v_tm_key_day = 20250401
# print(f'\nPostpaid Revenue...\n')
# print(f'   -> v_postpaid_metric_list: {v_postpaid_metric_list}')
# # print(f'   -> v_tm_key_day: {v_tm_key_day}')

# B2R000100_day_df = chk_src_df.copy()
# B2R000100_day_df = B2R000100_day_df.loc[B2R000100_day_df['METRIC_CD'].isin(v_postpaid_metric_list)]
# # B2R000100_day_df = B2R000100_day_df.loc[B2R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# # B2R000100_day_df = B2R000100_day_df.loc[B2R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

# B2R000100_day_df = B2R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
# B2R000100_day_df = B2R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

# B2R000100_day_df['Diff (C - P)'] = B2R000100_day_df['C'] - B2R000100_day_df['P']
# B2R000100_day_df = B2R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

# mod_col_list = B2R000100_day_df.iloc[:, 4:7].columns.tolist()
# for col in mod_col_list:
#     B2R000100_day_df[col] = B2R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# B2R000100_day_df

In [102]:
# ''' Postpaid Revenue B2C '''

# v_postpaid_b2c_metric_list = ['B2R010100', 'TB2R010100', 'DB2R010100']
# # v_tm_key_mth = 202501
# # v_tm_key_day = int(str(curr_mth)+str('01'))
# # v_tm_key_day = 20250401
# print(f'\nPostpaid Revenue B2C...\n')
# print(f'   -> v_postpaid_b2c_metric_list: {v_postpaid_b2c_metric_list}')
# # print(f'   -> v_tm_key_day: {v_tm_key_day}')

# B2R010100_day_df = chk_src_df.copy()
# B2R010100_day_df = B2R010100_day_df.loc[B2R010100_day_df['METRIC_CD'].isin(v_postpaid_b2c_metric_list)]
# # B2R010100_day_df = B2R010100_day_df.loc[B2R010100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# # B2R010100_day_df = B2R010100_day_df.loc[B2R010100_day_df['TM_KEY_DAY']>=v_tm_key_day]

# B2R010100_day_df = B2R010100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
# B2R010100_day_df = B2R010100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

# B2R010100_day_df['Diff (C - P)'] = B2R010100_day_df['C'] - B2R010100_day_df['P']
# B2R010100_day_df = B2R010100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

# mod_col_list = B2R010100_day_df.iloc[:, 4:7].columns.tolist()
# for col in mod_col_list:
#     B2R010100_day_df[col] = B2R010100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# B2R010100_day_df

In [92]:
''' TOL Revenue '''

v_tol_metric_list = ['TB3R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTOL Revenue...\n')
print(f'   -> v_tol_metric_list: {v_tol_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD'].isin(v_tol_metric_list)]
# TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

TB3R000100_day_df = TB3R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

TB3R000100_day_df['Diff (C - P)'] = TB3R000100_day_df['C'] - TB3R000100_day_df['P']
TB3R000100_day_df = TB3R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = TB3R000100_day_df.iloc[:, 4:7].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TOL Revenue...

   -> v_tol_metric_list: ['TB3R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,TOL,TB3R000100,TOL Revenue,"4,545,013","1,597,056,733","1,592,511,720",1.592512e+09,2025-05-02 15:37:10
1,202502,TOL,TB3R000100,TOL Revenue,"4,254,776","1,609,673,918","1,605,419,142",1.605419e+09,2025-05-02 15:37:10
2,202503,TOL,TB3R000100,TOL Revenue,"4,185,979","1,616,442,724","1,612,256,745",1.612257e+09,2025-05-02 15:37:10
3,202504,TOL,TB3R000100,TOL Revenue,"5,682,584","1,615,247,143","1,609,564,559",1.609565e+09,2025-05-02 15:37:10
4,202505,TOL,TB3R000100,TOL Revenue,0,0,0,0.000000e+00,2025-05-02 15:37:10


In [93]:
''' TVS Revenue '''

v_tvs_metric_list = ['TB4R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTVS Revenue...\n')
print(f'   -> v_tvs_metric_list: {v_tvs_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD'].isin(v_tvs_metric_list)]
# TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

TB4R000100_day_df = TB4R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

TB4R000100_day_df['Diff (C - P)'] = TB4R000100_day_df['C'] - TB4R000100_day_df['P']
TB4R000100_day_df = TB4R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = TB4R000100_day_df.iloc[:, 4:7].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TVS Revenue...

   -> v_tvs_metric_list: ['TB4R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,TVS,TB4R000100,TVS Revenue,"124,433,738","417,316,338","292,882,600",2.928826e+08,2025-05-02 15:37:10
1,202502,TVS,TB4R000100,TVS Revenue,"14,211,771","307,227,824","293,016,052",2.930161e+08,2025-05-02 15:37:10
2,202503,TVS,TB4R000100,TVS Revenue,"-82,621,243","214,617,965","297,239,208",2.972392e+08,2025-05-02 15:37:10
3,202504,TVS,TB4R000100,TVS Revenue,"11,465,481","216,776,308","205,310,827",2.053108e+08,2025-05-02 15:37:10
4,202505,TVS,TB4R000100,TVS Revenue,0,0,0,0.000000e+00,2025-05-02 15:37:10
